## Description:

This notebook aims to use the data within the `IAQF_DATA.csv` file to create the master dataframe for the competition. 

In [ ]:
import pandas as pd
from scipy.stats import norm
from math import log, sqrt, exp
import numpy as np

def d_plus( x, K, tau, sigma, rfr ):
    
    return ( (log(x/K) + (rfr + 0.5*(sigma*sigma))*tau ) * (1/(sigma*sqrt(tau))) )

def d_minus( x, K, tau, sigma, rfr ):
    
    return ( d_plus( x, K, tau, sigma, rfr ) - sigma*sqrt(tau) )


def bs_call_price( x, K, tau, sigma, rfr ):
    
    d_1 = d_plus( x, K, tau, sigma, rfr )
    d_2 = d_minus( x, K, tau, sigma, rfr )
    
    return ( x*norm.cdf(d_1) - K*exp(-1.0*rfr*tau)*norm.cdf(d_2) )

def bs_put_price( x, K, tau, sigma, rfr ):
    
    d_1 = d_plus( x, K, tau, sigma, rfr )
    d_2 = d_minus( x, K, tau, sigma, rfr )
    
    return ( K*exp(-1.0*rfr*tau)*norm.cdf(-1.0*d_2) - x*norm.cdf(-1.0*d_1) )

def build_master_dataframe():
    
    all_df = pd.read_csv('./IAQF_DATA.csv')
    
    rates_df = pd.DataFrame()
    rates_df['Dates'] = all_df['Dates_rfrate']
    rates_df['rf_rate'] = all_df['rf_rate']

    del all_df['Dates_rfrate']
    del all_df['rf_rate']
    
    master = pd.merge(all_df, rates_df, how='inner', on=['Dates'])
    
    master['bs_call'] = np.vectorize(bs_call_price)(master.SP500, master.SP500, 
                                                    90/360, master.vol/100, master.rf_rate/100)
    master['bs_put'] = np.vectorize(bs_put_price)(master.SP500, master.SP500, 
                                                  90/360, master.vol/100, master.rf_rate/100)

    master['ma_60'] = master.SP500.rolling(window=60).mean()
    master['ma_120'] = master.SP500.rolling(window=120).mean()
    master['cross'] = master.ma_60 > master.ma_120
    master['straddle'] = master.bs_call + master.bs_put
    
    instruments = ['SP500', 'bs_call', 'bs_put', 'straddle']
    
    for elm in instruments:
        retStr = elm + "_ret"
        master[retStr] = master[elm].pct_change()
        
    return master

## Let's do portfolio construction for here now .... then figure out how to use one python function in a separate jupyter notebook 

In [ ]:
master = build_master_dataframe()

In [ ]:
len(master)

In [ ]:
master = master[119:] # remove NaN values

In [ ]:
master.tail()

In [ ]:
port1 = master[['Dates','SP500','ma_60','ma_120','cross','SP500_ret']]

In [ ]:
port1.head()

In [ ]:
port1['port_ret'] = pd.Series()